# Imports

In [1]:
from numpy.random import seed
seed(888)
from tensorflow.compat.v1 import set_random_seed
set_random_seed(404)

In [5]:
import os
import numpy as np
import tensorflow as tf

# Constants

In [6]:
x_train_path='handwriting/digit_xtrain.csv'
y_train_path='handwriting/digit_ytrain.csv'
x_test_path='handwriting/digit_xtest.csv'
y_test_path='handwriting/digit_ytest.csv'

# Get the Data

In [8]:
%%time
x_train_all=np.loadtxt(x_train_path, delimiter=',', dtype=int)

Wall time: 17.5 s


In [9]:
%%time
x_test_all=np.loadtxt(x_test_path, delimiter=',', dtype=int)
y_train_all=np.loadtxt(y_train_path, delimiter=',', dtype=int)
y_test_all=np.loadtxt(y_test_path, delimiter=',', dtype=int)

Wall time: 3.1 s


In [10]:
y_train_all.shape

(60000,)